In [ ]:
import os 
import numpy as np
import pickle 
from tensorflow import keras 
import cv2
import random
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
data = []
img_size = 224
categories = ["daisy", "dandelion", "rose", "sunflower", "tulip"]
def create_data():
  for category in categories:
    path = os.path.join('/content/gdrive/My Drive/kaggle/Flowers classifier/flowers', category)
    class_num = categories.index(category)
    for img in os.listdir(path):
      img_arr = cv2.imread(os.path.join(path, img))
      try:
        new_arr = cv2.resize(img_arr, (img_size, img_size))
      except cv2.error as e:
        print('Not valid')
      cv2.waitKey()
        

      data.append([new_arr, class_num])
  


In [ ]:
create_data()
random.shuffle(data)
X=[]
y=[]

for features, labels in data:
  X.append(features)
  y.append(labels)

X = np.array(X).reshape(-1, img_size, img_size, 3)
y = np.array(y)

Not valid
Not valid
Not valid


In [ ]:
pickle_out = open('X_224.pickle', 'wb')

pickle.dump(X, pickle_out)
pickle_out.close()
pickle_out_2 = open('y_224.pickle', 'wb')

pickle.dump(y, pickle_out_2)
pickle_out.close()

In [ ]:
os.chdir('/content/gdrive/My Drive/kaggle/Flowers classifier')
pickle_in1 = open('X_224.pickle', 'rb')

X = pickle.load(pickle_in1)

pickle_in2 = open('y_224.pickle', 'rb')

y = pickle.load(pickle_in2)

In [ ]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
le = LabelEncoder()
y= le.fit_transform(y)
y = to_categorical(y,5)
y.shape


(4326, 5)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size = 0.8, random_state= 7)


In [ ]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False) 
datagen.fit(X_train)
X_train.shape

(3460, 224, 224, 3)

In [ ]:
model.compile(optimizer = keras.optimizers.Adam(learning_rate=0.001), loss = 'categorical_crossentropy', metrics=['accuracy'])



In [ ]:
from keras.applications.vgg16 import VGG16

vgg = VGG16(input_shape=X_train.shape[1:], weights = 'imagenet', include_top = False)

58892288/58889256 [==============================] - 2s 0us/step


In [ ]:
for layer in vgg.layers:
  layer.trainable = False

In [ ]:
x = keras.layers.Flatten()(vgg.output)

predictions = keras.layers.Dense(5, activation='softmax')(x)

model = keras.Model(inputs = vgg.input, outputs = predictions)

In [ ]:
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)      

In [ ]:
history = model.fit_generator(datagen.flow(X_train, y_train, batch_size=128), epochs=50, validation_data=(X_valid, y_valid))

Epoch 1/50
28/28 [==============================] - 44s 2s/step - loss: 7.7457 - accuracy: 0.6448 - val_loss: 4.7359 - val_accuracy: 0.7921
Epoch 2/50
28/28 [==============================] - 39s 1s/step - loss: 4.1168 - accuracy: 0.8003 - val_loss: 4.1757 - val_accuracy: 0.8129
Epoch 3/50
28/28 [==============================] - 38s 1s/step - loss: 3.3402 - accuracy: 0.8217 - val_loss: 4.0483 - val_accuracy: 0.8430
Epoch 4/50
28/28 [==============================] - 38s 1s/step - loss: 3.3254 - accuracy: 0.8327 - val_loss: 4.6174 - val_accuracy: 0.8256
Epoch 5/50
28/28 [==============================] - 38s 1s/step - loss: 3.0452 - accuracy: 0.8526 - val_loss: 4.5746 - val_accuracy: 0.8233
Epoch 6/50
28/28 [==============================] - 38s 1s/step - loss: 2.9250 - accuracy: 0.8604 - val_loss: 4.3536 - val_accuracy: 0.8499
Epoch 7/50
28/28 [==============================] - 40s 1s/step - loss: 2.6428 - accuracy: 0.8647 - val_loss: 4.1392 - val_accuracy: 0.8557
Epoch 8/50
28/28 [==